In [1]:
# Imports
import cv2
import os
from tqdm import tqdm
import numpy as np

In [2]:
# Settings
DIRECTORY = "Battlers"
MAX_WIDTH = 8192

OUT_DIRECTORY = "Results"

In [3]:
# Slice Function
def SliceImage(img, img_data):
    frames_in_line = (MAX_WIDTH // img_data['height'])
    slice_point = frames_in_line * img_data['height']
    remaining_size = img_data['width'] - slice_point
    # print("Will slice image: " + img_data['name'] + " at point: " + str(slice_point) + " and the remaining size is: " + str(remaining_size))
    
    first_part = img[:,:slice_point,:]
    second_part = img[:,slice_point:,:]
    
    # Second slice needed
    if (len(second_part[0]) > MAX_WIDTH):
        third_part = second_part[:,slice_point:,:]
        second_part = second_part[:,:slice_point,:]
        
        # Third slice needed
        if (len(third_part[0]) > MAX_WIDTH):
            # print("Third slice needed")
            fourth_part = third_part[:,slice_point:,:]
            third_part = third_part[:,:slice_point,:]
            
            # Fourth slice needed
            if (len(fourth_part[0]) > MAX_WIDTH):
                fifth_part = fourth_part[:,slice_point:,:]
                fourth_part = fourth_part[:,:slice_point,:]
                
                if(len(fifth_part[0]) > MAX_WIDTH):
                    sixth_part = fifth_part[:,slice_point:,:]
                    fifth_part = fifth_part[:,:slice_point,:]
                    
                    last_remaining_size = slice_point - (slice_point - len(sixth_part[0]))

                    new_img = np.zeros((6 * img_data['height'], len(first_part[0]), 4))
                    new_img[:img_data['height'],:slice_point,:] = first_part
                    new_img[img_data['height']:2 * img_data['height'],:remaining_size,:] = second_part
                    new_img[2 * img_data['height']:3 * img_data['height'],:remaining_size,:] = third_part
                    new_img[3 * img_data['height']:4 * img_data['height'],:remaining_size,:] = fourth_part
                    new_img[4 * img_data['height']:5 * img_data['height'],:remaining_size,:] = fifth_part
                    new_img[5 * img_data['height']:,:last_remaining_size,:] = sixth_part
                # Four slices needed
                else:
                    last_remaining_size = slice_point - (slice_point - len(fifth_part[0]))

                    new_img = np.zeros((5 * img_data['height'], len(first_part[0]), 4))
                    new_img[:img_data['height'],:slice_point,:] = first_part
                    new_img[img_data['height']:2 * img_data['height'],:remaining_size,:] = second_part
                    new_img[2 * img_data['height']:3 * img_data['height'],:remaining_size,:] = third_part
                    new_img[3 * img_data['height']:4 * img_data['height'],:remaining_size,:] = fourth_part
                    new_img[4 * img_data['height']:,:last_remaining_size,:] = fifth_part
            # Three slices needed
            else:
                last_remaining_size = slice_point - (slice_point - len(fourth_part[0]))

                new_img = np.zeros((4 * img_data['height'], len(first_part[0]), 4))
                new_img[:img_data['height'],:slice_point,:] = first_part
                new_img[img_data['height']:2 * img_data['height'],:remaining_size,:] = second_part
                new_img[2 * img_data['height']:3 * img_data['height'],:remaining_size,:] = third_part
                new_img[3 * img_data['height']:,:last_remaining_size,:] = fourth_part
        # Two slices needed
        else:
            last_remaining_size = slice_point - (slice_point - len(third_part[0]))

            new_img = np.zeros((3 * img_data['height'], len(first_part[0]), 4))
            new_img[:img_data['height'],:slice_point,:] = first_part
            new_img[img_data['height']:2 * img_data['height'],:remaining_size,:] = second_part
            new_img[2 * img_data['height']:,:last_remaining_size,:] = third_part
    # Only one slice needed
    else:
        new_img = np.zeros((2 * img_data['height'], len(first_part[0]), 4))
        new_img[:img_data['height'],:slice_point,:] = first_part
        new_img[img_data['height']:,:remaining_size,:] = second_part
    
    # print(len(new_img), len(new_img[0]), len(new_img[0][0]))
    
    return new_img

In [8]:
# Script
image_data = []
images = []

filenames = os.listdir(DIRECTORY)
filenames.sort()

for i in tqdm(range(len(filenames)), desc="Reading images"):
    filename = filenames[i]
    if (filename.endswith(".png")):
        data = {}
        data['name'] = filename
        data['image'] = cv2.imread(os.path.join(DIRECTORY, filename), cv2.IMREAD_UNCHANGED)
        images.append(data)
    
for i in range(len(images)):
    data = {}
    data['name'] = images[i]['name']
    data['height'] = len(images[i]['image'])
    data['width'] = len(images[i]['image'][0])
    image_data.append(data)
    
for i in tqdm(range(len(images)), desc="Slicing images"):
    if (image_data[i]['width'] > MAX_WIDTH):
        images[i]['image'] = SliceImage(images[i]['image'], image_data[i])
        
for i in tqdm(range(len(images)), desc="Writing images"):
    filename = os.path.join(OUT_DIRECTORY, image_data[i]['name'])
    cv2.imwrite(filename, images[i]['image'])

Writing images: 100%|██████████| 3324/3324 [01:54<00:00, 29.00it/s] 


In [15]:
# Info on special cases
special_directory = "Special Cases"

# Script
image_data = []
images = []

filenames = os.listdir(special_directory)
filenames.sort()

for i in tqdm(range(len(filenames)), desc="Reading images"):
    filename = filenames[i]
    if (filename.endswith(".png")):
        data = {}
        data['name'] = filename
        data['image'] = cv2.imread(os.path.join(special_directory, filename))
        images.append(data)

for i in range(len(images)):
    data = {}
    data['name'] = images[i]['name']
    data['height'] = len(images[i]['image'])
    data['width'] = len(images[i]['image'][0])
    image_data.append(data)
    print(data)

Reading images: 100%|██████████| 2/2 [00:00<00:00, 697.37it/s]

{'name': '575b.png', 'height': 59, 'width': 9381}


In [16]:
159 - 138

21